# MaxMind mmdb to csv converter
Can run directly in Google Colab - no VMs required

## Steps
1. Open this notebook in Google Colab
2. upload your mmdb to your Google Drive
3. start the conversion

In [ ]:
!pip install maxminddb==2.6.3

### Get from Google Drive
- Connect your Google Drive to Colab
- Upload your mmdb file

In [2]:
import os

path = "/content/drive/MyDrive/geoip_convert/"
mmdb_file = "GeoIP2-City-Test.mmdb"

os.environ["MMDB_DIR"] = path
os.environ["MMDB_FILE"] = mmdb_file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Or download directly here

In [ ]:
!wget --directory-prefix=$MMDB_DIR https://github.com/maxmind/MaxMind-DB/raw/main/test-data/$MMDB_FILE

In [ ]:
!ls $MMDB_DIR

In [11]:
import maxminddb
import ipaddress
import copy
import pandas as pd

Input the file name

In [5]:
output_file = 'out.csv'

In [6]:
counter=0
write_header = True

row_format = {
  'type': "ipv4",
  'range': "",
  'continent_code': "",
  'continent': "",
  'country_code': "",
  'country': "",
  'city': "",
  'region': "",
  'region_code': "",
  'latitude': None,
  'longitude': None,
  'location_accuracy_radius': None,
}
out_file = path + output_file

In [13]:
with maxminddb.open_database(path + mmdb_file) as reader:

  rows = []
  for node in reader:

    row = copy.deepcopy(row_format)

    row['range'] = format(node[0])

    if isinstance(node[0], ipaddress.IPv6Network):
      row['type'] = 'ipv6'

    d = node[1]

    if 'continent' in d:
      if 'code' in d['continent']:
        row['continent_code'] = d['continent']['code']

      if 'names' in d['continent']:
        if 'en' in d['continent']['names']:
            row['continent'] = d['continent']['names']['en']

    if 'registered_country' in d:
      if 'iso_code' in d['registered_country']:
          row['country_code'] = d['registered_country']['iso_code']

      if 'names' in d['registered_country']:
        if 'en' in d['registered_country']['names']:
            row['country'] = d['registered_country']['names']['en']

    if 'city' in d:
      if 'names' in d['city']:
          if 'en' in d['city']['names']:
            row['city'] = d['city']['names']['en']



    if 'subdivisions' in d:
      if 'names' in d['subdivisions'][0]:
          if 'en' in d['subdivisions'][0]['names']:
            row['region'] = d['subdivisions'][0]['names']['en']
      if 'names' in d['subdivisions'][0]:
            row['region_code'] = d['subdivisions'][0]['iso_code']


    if 'location' in d:
      if 'latitude' in d['location']:
        row['latitude'] = d['location']['latitude']

      if 'longitude' in d['location']:
        row['longitude'] = d['location']['longitude']

      if 'accuracy_radius' in d['location']:
        row['location_accuracy_radius'] = d['location']['accuracy_radius']

    if 'longitude' in d:
      row['longitude'] = d['longitude']

    counter += 1
    rows.append(row)

    if counter % 10000 == 0:
      pd.DataFrame(rows).to_csv(out_file, mode='a', header=write_header, index=False)
      write_header = False
      rows = []
      print('.',end='')

    if counter % 1000000 == 0:
      print('.')


pd.DataFrame(rows).to_csv(out_file, mode='a', header=False, index=False)



Check output

In [ ]:
!head "$MMDB_PATH/out.csv"